In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
#importing the training data
train_data=pd.read_csv('C:/Users/sdandir/Desktop/projects/NLP/imdb_train.csv')
test_data=pd.read_csv('C:/Users/sdandir/Desktop/projects/NLP/imdb_test_data.csv')

In [3]:
# Model configuration
max_sequence_length = 100
num_distinct_words = 10000
embedding_output_dims = 15
loss_function = 'binary_crossentropy'
optimizer = 'adam'
additional_metrics = ['accuracy']
number_of_epochs = 100
verbosity_mode = True
validation_split = 0.20

In [5]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [6]:
train_data.head()

,review,sentiment
0,We had STARZ free weekend and I switched on th...,negative
1,I'll admit that this isn't a great film. It pr...,negative
2,I finally found a version of Persuasion that I...,positive
3,The BBC surpassed themselves with the boundari...,positive
4,"Much praise has been lavished upon Farscape, b...",negative


In [7]:
from sklearn.preprocessing import LabelBinarizer
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
train_data['sentiment']=lb.fit_transform(train_data['sentiment'])


In [8]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

train_data['Processed_Reviews'] = train_data.review.apply(lambda x: clean_text(x))
test_data['Processed_Reviews'] = test_data.review.apply(lambda x: clean_text(x))

In [9]:
train_data.head()

,review,sentiment,Processed_Reviews
0,We had STARZ free weekend and I switched on th...,0,starz free weekend switch station see wa wa m...
1,I'll admit that this isn't a great film. It pr...,0,ill admit isnt great film practically scream l...
2,I finally found a version of Persuasion that I...,1,finally find version persuasion like anne does...
3,The BBC surpassed themselves with the boundari...,1,bbc surpass boundary cross tip velvet past the...
4,"Much praise has been lavished upon Farscape, b...",0,much praise ha lavish upon farscape dont think...


In [10]:
train_data.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()
test_data.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

124.127

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers


In [12]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train_data['Processed_Reviews'])

#add test in final model

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train_data['sentiment']

In [13]:
#creating model
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/3
320/320 [==============================] - 80s 237ms/step - loss: 0.5102 - accuracy: 0.7566 - val_loss: 0.2915 - val_accuracy: 0.8781
Epoch 2/3
320/320 [==============================] - 68s 212ms/step - loss: 0.2369 - accuracy: 0.9071 - val_loss: 0.2845 - val_accuracy: 0.8815
Epoch 3/3
320/320 [==============================] - 69s 216ms/step - loss: 0.1856 - accuracy: 0.9337 - val_loss: 0.3166 - val_accuracy: 0.8737


In [14]:
t_d=pd.read_csv('C:/Users/sdandir/Desktop/projects/NLP/imdb_test_data.csv')
t_d.head()

t_d = t_d.drop(['id'], axis=1)
t_d.head()

#t_d["review"]=test_data.review.apply(lambda x: clean_text(x))
#test_data["sentiment"] = test_data["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)

#t_d["review"]=test_data.review.apply(lambda x: len(x.split(" "))).mean()
#y_test = test_data["sentiment"]
#list_sentences_test = t_d["review"]
#list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
#X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
#prediction = model.predict(X_te)
#y_pred = (prediction > 0.5)
#from sklearn.metrics import f1_score, confusion_matrix
#print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
#print('Confusion matrix:')
#confusion_matrix(y_pred, y_test)

,review
0,Not only is this movie a great film for basic ...
1,"Waitress: Honey, here's them eggs you ordered...."
2,Many mystery stories follow the standard whodu...
3,A space ship cruising through the galaxy encou...
4,My favorite film this year. Great characters a...


In [15]:
t_d['Processed_Reviews'] = t_d.review.apply(lambda x: clean_text(x))

In [16]:
t_d.head()

,review,Processed_Reviews
0,Not only is this movie a great film for basic ...,movie great film basic cinematography screenpl...
1,"Waitress: Honey, here's them eggs you ordered....",waitress honey egg order honey like bee get th...
2,Many mystery stories follow the standard whodu...,many mystery story follow standard whodunit pa...
3,A space ship cruising through the galaxy encou...,space ship cruise galaxy encounter mysterious ...
4,My favorite film this year. Great characters a...,favorite film year great character plot direct...


In [17]:
t_d.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

124.127

In [18]:
list_sentences_test = t_d["Processed_Reviews"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)

In [19]:
y_pred = (prediction > 0.5)
print(y_pred)

[[False]
 [False]
 [ True]
 ...
 [False]
 [False]
 [False]]


In [20]:
predictions = pd.DataFrame()
predictions.insert(loc = 0 , column='id',value=test_data['id'])
predictions.insert(loc=1, column='sentiment',value=y_pred)

In [21]:
predictions['sentiment']=predictions['sentiment'].map({False:'negative', True:'positive'})
predictions.head()

,id,sentiment
0,1,negative
1,2,negative
2,3,positive
3,4,negative
4,5,positive


In [ ]:
predictions.to_csv('thirdsub.csv')